# Model Training
This notebook trains the XGBoost (Baseline) and MLP (Hybrid) models used for Pokemon Type Prediction.

In [1]:
import sys
import os
from pathlib import Path
if str(Path.cwd().parent) not in sys.path:
    sys.path.append(str(Path.cwd().parent))
if str(Path.cwd()) not in sys.path:
    sys.path.append(str(Path.cwd()))

import joblib
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.multioutput import MultiOutputClassifier
from xgboost import XGBClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.preprocessing import MultiLabelBinarizer

from pokemon_predictor import config
from pokemon_predictor.data_utils import load_data
from pokemon_predictor.model_utils import FocalLoss

## Train XGBoost

In [2]:
def train_xgboost():
    print("Training XGBoost Model...")
    X_train, X_test, y_train, y_test, classes = load_data('rgb', split_data=True)
    
    mlb = MultiLabelBinarizer()
    mlb.classes_ = classes
    
    model = MultiOutputClassifier(XGBClassifier(n_estimators=100, max_depth=5, learning_rate=0.1, colsample_bytree=0.75, n_jobs=-1, random_state=42))
    model.fit(X_train, y_train)
    
    score = model.score(X_test, y_test)
    print(f"XGBoost Test Accuracy (Subset): {score:.4f}")
    
    out_path = config.MODELS_DIR / "xgboost_model.pkl"
    joblib.dump(model, out_path)
    print(f"Saved XGBoost model to {out_path}")
    
    y_labels = pd.read_csv(config.PROCESSED_DATA_DIR / "y_labels.csv")
    y_list = []
    for _, row in y_labels.iterrows():
        types = [row['type1']]
        if pd.notna(row['type2']):
            types.append(row['type2'])
        y_list.append(types)
    mlb_full = MultiLabelBinarizer()
    mlb_full.fit(y_list)
    joblib.dump(mlb_full, config.MODELS_DIR / "mlb.pkl")
    print(f"Saved MLB to {config.MODELS_DIR / 'mlb.pkl'}")

train_xgboost()

Training XGBoost Model...


XGBoost Test Accuracy (Subset): 0.2485
Saved XGBoost model to /Users/yasha/Desktop/projects/pokemon_type_predictor/models/xgboost_model.pkl
Saved MLB to /Users/yasha/Desktop/projects/pokemon_type_predictor/models/mlb.pkl


## Train MLP

In [3]:
def train_mlp():
    print("\nTraining MLP Model (Hybrid)...")
    X_train, X_test, y_train, y_test, classes = load_data('hybrid', split_data=True)
    
    model = Sequential([
        Input(shape=(X_train.shape[1],)),
        Dense(512, activation='relu'),
        BatchNormalization(),
        Dropout(0.4),
        Dense(256, activation='relu'),
        BatchNormalization(),
        Dropout(0.3),
        Dense(len(classes), activation='sigmoid')
    ])
    
    model.compile(optimizer=Adam(0.001), loss=FocalLoss(), metrics=['accuracy', 'binary_accuracy'])
    
    early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    
    history = model.fit(
        X_train, y_train,
        validation_data=(X_test, y_test),
        epochs=50,
        batch_size=32,
        callbacks=[early_stop],
        verbose=1
    )
    
    out_path = config.MODELS_DIR / "mlp_model_optimized.h5"
    model.save(out_path)
    print(f"Saved MLP model to {out_path}")
    
    print("Finding best threshold...")
    y_pred = model.predict(X_test)
    best_thresh = 0.5
    best_f1 = 0.0
    
    from sklearn.metrics import f1_score
    for thresh in np.arange(0.1, 0.9, 0.05):
        y_pred_bin = (y_pred > thresh).astype(int)
        f1 = f1_score(y_test, y_pred_bin, average='micro')
        if f1 > best_f1:
            best_f1 = f1
            best_thresh = thresh
            
    print(f"Best Threshold: {best_thresh:.2f} (F1: {best_f1:.4f})")
    joblib.dump(best_thresh, config.MODELS_DIR / "best_threshold.pkl")

train_mlp()


Training MLP Model (Hybrid)...


Epoch 1/50


 1/83 ━━━━━━━━━━━━━━━━━━━━ 51s 629ms/step - accuracy: 0.1562 - binary_accuracy: 0.4878 - loss: 0.1922

27/83 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0594 - binary_accuracy: 0.4912 - loss: 0.1989   

56/83 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0578 - binary_accuracy: 0.4873 - loss: 0.1910

...

83/83 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0611 - binary_accuracy: 0.0846 - loss: 2.6723e-05 - val_accuracy: 0.0530 - val_binary_accuracy: 0.0848 - val_loss: 1.1228e-06


Saved MLP model to /Users/yasha/Desktop/projects/pokemon_type_predictor/models/mlp_model_optimized.h5
Finding best threshold...


 1/21 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step

21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 


Best Threshold: 0.80 (F1: 0.1563)
